# 05 - Statistiques Univariées

In [ ]:
import os # Vérif du chemin d'accès à retirer après
os.getcwd() # Vérif du chemin d'accès à retirer après



> ### Comment calculer la moyenne de nos températures à la surface dans le cas de la Manche ?
Tout d'abord, rappelons la formule globale pour le calcul d'une moyenne spatiale sur une sphère :

$$
\bar{T}(t) = \frac{
\sum_{\text{lat, lon}} T(t, \text{lat}, \text{lon}) \cdot \cos(\text{lat})
}{
\sum_{\text{lat, lon}} \cos(\text{lat})
}
$$

Les termes $cos(lat)$ permettent de faire la pondération à cause de la déformation spatiale d'une sphère : un pixel au niveau des pôles aura une pondération plus faible qu'un pixel au niveau de l'équateur. Cette pondération ne prend pas en compte la longitude puisque celle-ci est fixe pour une latitude donnée.

Cependant dans notre cas nous sommes dans une zone géographique réduite (la Manche). Par conséquent la différence de surface représentée par nos pixels est négligeable et nous pouvons donc simplifier la formule :

$$
\bar{T}(t) = \frac{
\sum_{\text{lat, lon}} T(t, \text{lat}, \text{lon})
}{
N_{\text{pixels}}
}
$$

Ainsi nous obtenons un vecteur temporel contenant la moyenne des températures à la surface de la Manche chaque jour.



In [ ]:
import xarray as xr
import matplotlib.pyplot as plt

dataset = xr.open_dataset("../../data/processed/sstCOPERNICUS20102019Processed.nc")

sst_mean = dataset.analysed_sst.mean(dim=["latitude", "longitude"])

print(sst_mean)

sst_mean.plot()

> ### Comment calculer la valeur médiane dans notre dataset ?
&nbsp; 
La médiane spatiale pour un jour donné correspond à la valeur de température qui sépare la moitié des pixels océaniques de la Manche ayant une valeur supérieure et l'autre moitié ayant une valeur inférieure.
Pour un dataset global sur une sphère il faudrait pondérer les pixels par leur surface avec $\cos(lat)$ puis cumuler les poids jusqu'à atteindre 50% du total.
Cependant pour les mêmes raisons que notre moyenne nous pouvons donc ignorer les poids. 
Ainsi pour chaque jour nous prenons toutes les valeurs SST des pixels puis nous calculons la médiane spatiale c'est-à-dire la valeur centrale de l’échantillon ordonné. Avec 3652 valeurs (une par jour sur 9 ans) nous obtenons alors un vecteur temporel contenant la médiane SST de la Manche pour chaque jour.

In [ ]:
sst_median = dataset.analysed_sst.median(dim=["latitude","longitude"])

print(sst_median)
sst_median.plot()

> ### Comment calculer la variance dans notre dataset ?

La variance sert à mesurer la dispersion des valeurs autour de la moyenne, la formule de la variance spatiale dans le cas d'une sphère est :
$$
Var(t) = \frac{
\sum_{\text{lat, lon}} (T(t, \text{lat}, \text{lon}) - \bar{T}(t))^2 \cdot \cos(\text{lat})
}{
\sum_{\text{lat, lon}} \cos(\text{lat})
}
$$
où :
- $T(t, \text{lat}, \text{lon})$ est la SST d'un pixel
- $\bar{T}(t)$ est la moyenne spatiale à l'instant t
- $\cos(lat)$ pondère chaque pixel par sa surface réelle

Cette formulation est la généralisation spatiale de la variance classique.

Puisque dans notre étude nous sommes dans une zone géographique limitée et que nos données terrestres et NaN ont déjà été supprimées alors on peut simplifier la formule en :
$$
Var(t) = \frac{1}{N_{\text{pixels}}}
\sum_{\text{lat, lon}} (T(t, \text{lat}, \text{lon})-\bar{T}(t))^2
$$
Cette variance mesure donc chaque jour la variabilité spatiale de la SST dans la Manche.

In [ ]:
sst_variance = dataset.analysed_sst.var(dim=["latitude", "longitude"])
print(sst_variance)
sst_variance.plot()
plt.show()


>### Calcul de l'écart-type pour la température à la surface de la Manche
L'écart-type est défini comme étant la racine carrée de la variance. Pour une étude spatiale sur une sphère la formule générale est :
$$
\sigma(t) = \sqrt{ \frac{
\sum_{\text{lat, lon}} (T(t, \text{lat}, \text{lon}) - \bar{T}(t))^2 \cdot \cos(\text{lat})
}{
\sum_{\text{lat, lon}} \cos(\text{lat})
}}
$$
avec $T$ la température désaisonnalisée, $\bar{T}$ la moyenne spatiale des anomalies et $\cos{(\text{lat})}$ la pondération surfacique.
Cependant lorsqu'on adapte la formule à notre cas, on peut simplifier l'équation en :
$$
\sigma(t) = \sqrt{ \frac{1}{N_{\text{pixels}}}
\sum_{\text{lat, lon}} (T(t, \text{lat}, \text{lon}) - \bar{T}(t))^2}
$$
Cet écart-type représente donc la variabilité spatiale des anomalies SST dans la Manche pour chaque jour, indépendamment du cycle saisonnier.

> ### Pourquoi utiliser des données désaisonnalisées ?
Sur nos données brutes l'écart-type sera fortement influencé par le cycle annuel. En utilisant des données désaisonnalisées on peut mesurer uniquement la variabilité résiduelle liée aux évènements océaniques, atmosphériques ou dynamiques locales

In [ ]:
dataset2 = xr.open_dataset("../../data/processed/sstDeseasonalizedCOPERNICUS20102019.nc")
sst_variance = dataset2.analysed_sst.std(dim=["latitude","longitude"])
print(sst_variance)
sst_variance.plot()
plt.show()

> ### Comment calculer le minimum et le maximum de notre jeu de données ?

Pour un calcul de température spatiale sur une sphère, les extremes globaux sont définis par :
$$
\text{Minimum spatial : }
T_{\min}(t) = \min_{\text{lat, lon}} \left( T(t,\text{lat},\text
{lon}) \right)
$$
$$
\text{Maximum spatial : }
T_{\max}(t) = \max_{\text{lat, lon}} \left( T(t,\text{lat},\text
{lon}) \right)
$$
Ces deux valeurs représentent physiquement l'anomalie la plus froide et la plus chaude présente à la surface de la Manche quotidiennement.